In [1]:
import torch
from torchvision.models.resnet import resnet18
from wilds import get_dataset
from wilds.common.data_loaders import get_eval_loader, get_train_loader
from wilds.common.grouper import CombinatorialGrouper

import torchvision.transforms.v2 as T
from tqdm import tqdm

/home/yasin/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/yasin/anaconda3/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future chan

In [2]:
model = resnet18(num_classes=2)

In [3]:
transform = T.Compose([
    T.ToTensor()
])

dataset = get_dataset("camelyon17", root_dir="../../data/")
grouper = CombinatorialGrouper(dataset, ['hospital'])
train_set = dataset.get_subset("train", transform=transform)
train_loader = get_train_loader("standard", train_set, batch_size=32, num_workers=4)

/home/yasin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [4]:
NUM_EPOCHS = 5

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=1e-3, weight_decay=1e-2, momentum=0.9)

In [5]:
for i in range(NUM_EPOCHS):
    cum_loss = 0
    for i, (X, t, M) in enumerate(tqdm(train_loader)):

        optimizer.zero_grad()

        y = model(X)
        loss = criterion(y, t)

        loss.backward()
        optimizer.step()

        cum_loss += loss.item()

        if i % 20 == 19:
            print(f'Loss: {(cum_loss / 20):.4f}') 
            cum_loss = 0


  0%|          | 0/9452 [00:00<?, ?it/s]

  0%|          | 20/9452 [00:48<6:00:32,  2.29s/it]

Loss: 0.6330


  0%|          | 40/9452 [01:34<6:04:02,  2.32s/it]

Loss: 0.5227


...

Loss: 0.3136


  1%|▏         | 140/9452 [05:23<5:14:32,  2.03s/it]

Loss: 0.2887


  2%|▏         | 152/9452 [20:03<20:27:36,  7.92s/it]  


KeyboardInterrupt: 